In [1]:
import pandas as pd
import numpy as np

import neologdn
import MeCab

import re

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

C:\Users\Tomori kengo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### 日誌サンプルデータの読み込み

In [2]:
#encoding='utf-8'で上手くいった
diary_df = pd.read_csv('SBI_Financial statement_201903.csv', encoding='utf-8')

In [3]:
diary_df

,text
0,国内経済が緩やかに回復するなか、マーケット環境は、米国と中国の貿易摩擦問題に対する警戒感等か...
1,このような環境の中、当社業績においては、ホールセールビジネスの拡大、トレーディング収益や金融...
2,当社は引続き他社を大きく上回る高いシェアを維持し、35.9％のシェアを獲得。
3,先物・オプションの委託個人売買代金シェアは、引き続き高水準を維持。
4,投資信託残高の四半期末残高は過去最高を更新し、信託報酬は高水準を維持。
5,2018年4月から2018年6月までの上場会社数は20社。
6,同期間のSBI証券引受関与率は100％と 引き続き業界トップ。


### MeCabにかける前準備としてneologdn.normalize()を使用して文章全体を正規化

In [4]:
def get_diary_normalization(text):
    diary_normalization = neologdn.normalize(text)
    return diary_normalization

### MeCab + neologdで形態素解析し、名詞、形容詞原形と動詞原形を抽出しリストに格納
##### 参考：https://github.com/kujirahand/book-mlearn-gyomu/blob/master/src/ch4/Doc2Vec/create_model.py

In [5]:
neologd_tagger = MeCab.Tagger('-Ochasen -d C:\mecab-ipadic-neologd')

# 引数のテキストを分かち書きして配列にする
# node.surface: 文字のみ取得できる 出力例：同  期間  の  SBI  証券
# node.feature: 品詞、原形などの詳細を取得できる:「品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音」の順
# よって、「品詞」を取得したい場合はnode.featureをsplit()でリスト型にした後に抽出したい詳細のインデックス番号[0]を指定すればよい
# また、動詞や形容詞の「原形（の単語）」を取得したい場合はインデックス番号[6]を指定すればよい

def split_words(diary_normalization):
    node = neologd_tagger.parseToNode(diary_normalization) #parseだとエラー「 'str' object has no attribute 'feature'」
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if  hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["動詞", "形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words

In [6]:
wakati_list = []

# 文章の上から順に作成した関数を実行
for diary in diary_df['text']:
    diary_normalization = get_diary_normalization(diary)
    wakati_words = split_words(diary_normalization)
    wakati_list.append(wakati_words)
    
wakati_list

[['国内',
  '経済',
  '緩やか',
  '回復',
  'する',
  'なか',
  'マーケット',
  '環境',
  '米国',
  '中国',
  '貿易摩擦',
  '問題',
  '警戒感',
  '等',
  '一進一退',
  '展開'],
 ['よう',
  '環境',
  '中',
  '当社',
  '業績',
  'ホールセール',
  'ビジネス',
  '拡大',
  'トレーディング',
  '収益',
  '金融',
  '収益',
  '増加',
  '前年同期',
  '比',
  '増収増益',
  '達成'],
 ['当社', '他社', '大きい', '上回る', '高い', 'シェア', '維持', 'する', '35.9%', 'シェア', '獲得'],
 ['先物', 'オプション', '委託', '個人', '売買代金', 'シェア', '高水準', '維持'],
 ['投資信託', '残高', '四半期', '末', '残高', '過去最高', '更新', 'する', '信託報酬', '高水準', '維持'],
 ['2018年', '4月', '2018年', '6月', '上場会社', '数', '20', '社'],
 ['期間', 'SBI証券', '引受', '関与', '率', '100%', '業界', 'トップ']]

In [7]:
trainings_wakati_list = [TaggedDocument(words = wakati_list[i], tags = [i]) for i in range(len(diary_df))]
trainings_wakati_list

[TaggedDocument(words=['国内', '経済', '緩やか', '回復', 'する', 'なか', 'マーケット', '環境', '米国', '中国', '貿易摩擦', '問題', '警戒感', '等', '一進一退', '展開'], tags=[0]),
 TaggedDocument(words=['よう', '環境', '中', '当社', '業績', 'ホールセール', 'ビジネス', '拡大', 'トレーディング', '収益', '金融', '収益', '増加', '前年同期', '比', '増収増益', '達成'], tags=[1]),
 TaggedDocument(words=['当社', '他社', '大きい', '上回る', '高い', 'シェア', '維持', 'する', '35.9%', 'シェア', '獲得'], tags=[2]),
 TaggedDocument(words=['先物', 'オプション', '委託', '個人', '売買代金', 'シェア', '高水準', '維持'], tags=[3]),
 TaggedDocument(words=['投資信託', '残高', '四半期', '末', '残高', '過去最高', '更新', 'する', '信託報酬', '高水準', '維持'], tags=[4]),
 TaggedDocument(words=['2018年', '4月', '2018年', '6月', '上場会社', '数', '20', '社'], tags=[5]),
 TaggedDocument(words=['期間', 'SBI証券', '引受', '関与', '率', '100%', '業界', 'トップ'], tags=[6])]

In [8]:
# MUFGのFinancial Statementで学習したモデルのロード
model = Doc2Vec.load('MFUG_Financial_statement.model')

In [15]:
# 学習したモデルを使って、「今回の文書」同士の類似度を調べる
model.docvecs.similarity_unseen_docs(model, wakati_list[0], wakati_list[1], alpha=1, min_alpha=0.0001, steps=5)

0.5842480364298264

In [16]:
model.docvecs.similarity_unseen_docs(model, wakati_list[0], wakati_list[5], alpha=1, min_alpha=0.0001, steps=5)

0.39001291609943156

In [12]:
# 任意の文書間の類似度を調べる
# 引数は学習した文章(MFUG)のtagを指定
# このtagは今回の文章ではなく学習した文章(MFUG)のtag
model.docvecs.similarity(0,8)

0.16607250485776925

In [17]:
# 学習文書の中で、指定したidの文書と類似度の高い文書を調べる
# 引数は学習した文章(MFUG)のtagを指定
# このtagは今回の文章ではなく学習した文章のtag
model.docvecs.most_similar(0)

[(8, 0.1660725176334381),
 (9, 0.13208238780498505),
 (13, 0.1253616213798523),
 (5, 0.1136433333158493),
 (7, 0.10438272356987),
 (16, 0.10097911953926086),
 (2, 0.10084007680416107),
 (14, 0.0999135822057724),
 (11, 0.09619168192148209),
 (1, 0.08139412105083466)]

##### 参考：https://qiita.com/asian373asian/items/1be1bec7f2297b8326cf